In [2]:
m = 3
s = 10
t = 1
v = s / t 
a = v / t

F = m * a
F

30.0

In [29]:
import random
from functools import reduce

In [156]:
class Constant:
    def __init__(self, data, children = set(), op="", gradient = 0, label=""):
        self.data = data
        self.gradient = gradient
        self.children = children
        self.op = op
        self.backpropagate = lambda : None 
        self.label = label
    
    def __pow__(self, pow):
        out = Constant(self.data**pow, children=set((self, Constant(pow))), op=f"**{pow}")

        def backpropagate():
            self.gradient += out.gradient * (pow * (self.data ** (pow - 1)))
        
        out.backpropagate = backpropagate

        return out
    
    def __neg__(self):
        self.data = -self.data
        return self
    
    def __sub__(self, other):
        out = Constant(self.data - other.data, children=set((self, other)), op='-')

        def backpropagate():
            self.gradient += out.gradient 
            other.gradient += -out.gradient
    
        out.backpropagate = backpropagate
        return out
    
    def __add__(self, other):
        out = Constant(self.data + other.data, children=set((self, other)), op='+')

        def backpropagate():
            self.gradient += out.gradient 
            other.gradient += out.gradient
    
        out.backpropagate = backpropagate
        return out
    
    def __truediv__(self, other):
        out = Constant(self.data / other.data, children= set((self, other)), op="/")
        def backpropagate():
            self.gradient += out.gradient * (1 / other.data)
            other.gradient += out.gradient * - (self.data / (other.data ** 2))
        out.backpropagate = backpropagate
        return out
    
    def __mul__(self, other):
        out = Constant(self.data * other.data, children= set((self, other)), op='*')
        def backpropagate():
            self.gradient += out.gradient * other.data
            other.gradient += out.gradient * self.data
        out.backpropagate = backpropagate
        return out
    
    def backpropagate_recursive(self):
        self.gradient = 1

        def topological_sort(root: Constant)-> list[Constant]:
            data = []
            for child in list(root.children):
                data.extend(topological_sort(child))
            data.append(root)
            return data
        
        nodes = topological_sort(self)

        for node in reversed(nodes):
            node.backpropagate()
        
    
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        return f"Constant(label: {self.label}, Data:{self.data}, Gradient: {self.gradient})"

In [31]:
m = Constant(3, label="m")
s = Constant(10, label="s")
t = Constant(1, label="t")

v = s / t; v.label = "v"
a = v / t; a.label = "a"

F = m * a; F.label = "F"
F

Constant(label: F, Data:30.0, Gradient: 0)

In [32]:
F.gradient = 1
F.backpropagate_recursive()

In [33]:
def print_nodes(root: Constant, starting: bool = True):
    if starting: print(root)
    for child in root.children:
        print(child)
        
    for child in root.children:
        print_nodes(child, starting=False)

In [34]:
print_nodes(F)

Constant(label: F, Data:30.0, Gradient: 1)
Constant(label: m, Data:3, Gradient: 10.0)
Constant(label: a, Data:10.0, Gradient: 3)
Constant(label: v, Data:10.0, Gradient: 3.0)
Constant(label: t, Data:1, Gradient: -60.0)
Constant(label: s, Data:10, Gradient: 3.0)
Constant(label: t, Data:1, Gradient: -60.0)


In [69]:
class Neuron:
    def __init__(self, n_inputs):
        self.n = n_inputs
        self.bias = Constant(random.randint(-1, 1))
        self.w  = [Constant(random.uniform(-1, 1)) for i in range(self.n)]
    
    def parameters(self):
        return [self.bias] + self.w
    
    def compute(self, inputs):
        inputs = map(lambda x: x if isinstance(x, Constant) else Constant(x), inputs)
        return sum(map(lambda a: a[0] * a[1], zip(self.w, inputs)), self.bias)

In [70]:
n = Neuron(2)
data = [2, 3]
n.compute(data)

Constant(label: , Data:0.1246020019641012, Gradient: 0)

In [99]:
class Layer:
    def __init__(self, n_inputs, n_neurons):
        self.neurons = [ Neuron(n_inputs) for i in range(n_neurons) ]
    
    def parameters(self):
        p = []
        for neuron in self.neurons:
            p.extend(neuron.parameters())
        return p 
        
    def compute(self, inputs):
        return list(map(lambda x: x.compute(inputs), self.neurons))

In [82]:
inputs = [2, 2]

layer = Layer(2, 2)
layer.compute(inputs)

[Constant(label: , Data:3.175214228457051, Gradient: 0),
 Constant(label: , Data:-0.36958348101255645, Gradient: 0)]

In [94]:
class MultiLayerPerceptron:
    def __init__(self, n_inputs, n_layers):
        layer_sizes = [n_inputs]
        layer_sizes.extend(n_layers)
        self.layers = [Layer(layer_sizes[i], layer_sizes[i + 1]) for i in range(len(n_layers))]
    
    def parameters(self):
        p = []
        for layer in self.layers:
            p.extend(layer.parameters())
        return p 

    def compute(self, inputs):
        return reduce(lambda a,b: b.compute(a), self.layers, inputs)

In [131]:
# Creation of MLP
mlp = MultiLayerPerceptron(2, [3,4,1])

In [151]:
def lossfn(outputs: list[Constant], expected_output: list[int]) -> Constant:
    return sum(map(lambda a: (a[0] - Constant(a[1]))**2 ,zip(outputs, expected_output)), start=Constant(0))

In [166]:
# Gradient Descent
ip = [2, 3]
expected_output = [2]

while(1):
    output = mlp.compute(ip)

    ## Compute Loss
    loss = lossfn(output, expected_output)

    ## Break the loop is loss is within 0.01
    if loss.data <= 0.0015:
        break

    ## Reset Gradients
    for p in mlp.parameters():
        p.gradient = 0

    ## Backpropagate
    loss.backpropagate_recursive()


    ## Tweak Parameters
    learning_rate = 0.001

    for p in mlp.parameters():
        p.data += learning_rate * (-p.gradient)

output

[Constant(label: , Data:1.9710839082468654, Gradient: 0)]

In [167]:
loss

Constant(label: , Data:0.0008361403622756981, Gradient: 0)